<a href="https://colab.research.google.com/github/yeldos-umbetov/iBEC-robot/blob/main/Generate_text_with_RuGPTs_HF_torch_1_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate text with RuGPTs in huggingface
How to generate text with pretrained RuGPTs models with huggingface.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!pip3 install transformers==3.5.0
!pip install torch==1.4.0
# !pip install flask-ngrok
!pip install flask

     |████████████████████████████████| 1.3MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 38.4MB/s 
     |████████████████████████████████| 2.9MB 41.6MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 
     |████████████████████████████████| 753.4MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 613 (delta 68), reused 46 (delta 24), pack-reused 505
Receiving objects: 100% (613/613), 360.66 KiB | 2.98 MiB/s, done.
Resolving deltas: 100% (368/368), done.


## Generate

In [ ]:
import numpy as np
import torch

from flask import Flask, jsonify, request
# from flask_ngrok import run_with_ngrok

TRAIN_TEXT_FILE_PATH = "train_text_5133.txt"


In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

# check_text = "Компания: БургерКинг. Слоган: Лучшие бургеры на планете."
def generate(
    model, tok, text,
    do_sample=True, max_length=50, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=0.85,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

### RuGPT3Small

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")
generated = generate(model, tok, "Компания Новые двери. Слоган: ", num_beams=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated[0]

'Компания Новые двери. Слоган:  "Сделай свой дом лучше, чем он есть на самом деле"\nhttp://www.liveinternet.ru/users/sasha_korotchenko/post'

In [ ]:

app = Flask(__name__)
# run_with_ngrok(app)

@app.route("/")
def hello():
    return "Hello World!! from Google Colab"
@app.route("/bot")
def hellobot():
    #Компания: БургерКинг. Слоган: Лучшие бургеры на планете. Компания: 
    # split('"')[1::2]
    # "Компания: Tesla Inc. Индустрия: Автомобильная промышленность. Продукт: электромобиль. Слоган: "
    company = request.args.get('company')
    industry = request.args.get('industry')
    product = request.args.get('product')
    # mystr[mystr.find(char1)+1 : mystr.find(char2)]
    result_text = "Компания: " + company + ". Индустрия: " + industry + ". Слоган: " 
    
    tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")
    generated = generate(model, tok, result_text, num_beams=10)
    generated_text = generated[0]
    return generated_text
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
import requests

result = requests.get("http://3abaede87cbc.ngrok.io/bot?company=Эверест&industry=Реклама&product=Маркетинг")
print(result.text)

Tunnel 3abaede87cbc.ngrok.io not found


### RuGPT3Medium

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3medium_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
generated[0]

'Александр Сергеевич Пушкин родился в  1799 году. Его отец, Александр Сергеевич Пушкин, был генерал-адъютантом и воспитателем великого поэта Александра Сергеевича Грибоедова (1795–1829). В 1811 году он поступил'

### RuGPT3Large

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3large_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
generated[0]

'Александр Сергеевич Пушкин родился в \n1799 году. Его отец был крепостным крестьянином, а мать – крепостной крестьянкой. Детство и юность Пушкина прошли в деревне Михайловское под Петербургом. В 1820-х годах семья переехала'